1.Import necessary libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

# 核心库
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from typing import List, Tuple, Dict, Any, Optional
import os
import json
import time
from datetime import datetime

# TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow.keras import models, callbacks
    from tensorflow.keras.optimizers import Adam, SGD, RMSprop
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
except ImportError as e:
    print(f"TensorFlow import error: {e}")

# 设置
plt.style.use('ggplot')
sns.set_palette("husl")
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries imported successfully!")

2025-09-26 11:08:47.101632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758856127.109716   13698 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758856127.112196   13698 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-26 11:08:47.121811: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Libraries imported successfully!


2. Load Data and Models

In [2]:
def load_training_data(data_path: str = '../data/augmented') -> Tuple:
    """加载训练数据"""
    print("Loading training data...")
    
    x_train = np.load(os.path.join(data_path, 'x_train_aug.npy'))
    y_train = np.load(os.path.join(data_path, 'y_train_aug.npy'))
    x_val = np.load(os.path.join(data_path, 'x_val_aug.npy'))
    y_val = np.load(os.path.join(data_path, 'y_val_aug.npy'))
    
    # 转换为分类格式
    with open(os.path.join(data_path, 'class_names.txt'), 'r', encoding='utf-8') as f:
        class_names = [line.strip() for line in f.readlines()]
    
    num_classes = len(class_names)
    y_train_cat = to_categorical(y_train, num_classes)
    y_val_cat = to_categorical(y_val, num_classes)
    
    print(f"Training data: {x_train.shape} -> {y_train_cat.shape}")
    print(f"Validation data: {x_val.shape} -> {y_val_cat.shape}")
    
    return x_train, y_train_cat, x_val, y_val_cat, class_names, num_classes

def load_model_architecture(model_name: str, model_dir: str = '../models') -> models.Model:
    """加载模型架构"""
    model_path = os.path.join(model_dir, f'{model_name}_config.json')
    
    with open(model_path, 'r') as f:
        config = json.load(f)
    
    if 'class_name' in config:  # Functional API
        model = models.Model.from_config(config)
    else:  # Sequential API
        model = models.Sequential.from_config(config)
    
    print(f"Loaded {model_name} architecture")
    return model

# 加载数据
x_train, y_train, x_val, y_val, CLASS_NAMES, NUM_CLASSES = load_training_data()

# 加载模型
MODEL_NAMES = ['simple_cnn', 'advanced_cnn', 'lightweight_cnn']
models_dict = {}

for model_name in MODEL_NAMES:
    try:
        model = load_model_architecture(model_name)
        models_dict[model_name] = model
    except FileNotFoundError:
        print(f"Model {model_name} not found, skipping...")

print(f"Loaded {len(models_dict)} models for training")

Loading training data...
Training data: (40000, 32, 32, 3) -> (40000, 10)
Validation data: (10000, 32, 32, 3) -> (10000, 10)


I0000 00:00:1758856130.138692   13698 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3600 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Loaded simple_cnn architecture
Loaded advanced_cnn architecture
Loaded lightweight_cnn architecture
Loaded 3 models for training


3. Training Configuration

In [3]:
def create_training_configurations() -> List[Dict[str, Any]]:
    """创建不同的训练配置"""
    
    configurations = [
        {
            'name': 'adam_fast',
            'optimizer': 'adam',
            'learning_rate': 0.001,
            'batch_size': 64,
            'epochs': 50,
            'patience': 10
        },
        {
            'name': 'adam_slow',
            'optimizer': 'adam', 
            'learning_rate': 0.0001,
            'batch_size': 32,
            'epochs': 100,
            'patience': 15
        },
        {
            'name': 'sgd_momentum',
            'optimizer': 'sgd',
            'learning_rate': 0.01,
            'batch_size': 64,
            'epochs': 80,
            'patience': 12
        },
        {
            'name': 'rmsprop',
            'optimizer': 'rmsprop',
            'learning_rate': 0.001,
            'batch_size': 128,
            'epochs': 60,
            'patience': 10
        }
    ]
    
    return configurations

def create_callbacks(patience: int = 10, model_name: str = 'model') -> List:
    """创建训练回调函数"""
    
    callbacks_list = [
        # 早停
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=patience,
            restore_best_weights=True,
            verbose=1
        ),
        # 学习率调度
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=patience//2,
            min_lr=1e-7,
            verbose=1
        ),
        # 模型检查点
        callbacks.ModelCheckpoint(
            filepath=f'../models/{model_name}_best.h5',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        # TensorBoard
        callbacks.TensorBoard(
            log_dir=f'../logs/{model_name}_{datetime.now().strftime("%Y%m%d-%H%M%S")}',
            histogram_freq=1
        )
    ]
    
    return callbacks_list

# 创建训练配置
training_configs = create_training_configurations()
print(f"Created {len(training_configs)} training configurations")

Created 4 training configurations


4. Model Training Function

In [4]:
def compile_and_train_model(model: models.Model, 
                          x_train: np.ndarray, y_train: np.ndarray,
                          x_val: np.ndarray, y_val: np.ndarray,
                          config: Dict[str, Any],
                          model_name: str) -> Dict[str, Any]:
    """编译和训练模型"""
    
    print(f"\nTraining {model_name} with {config['name']} configuration...")
    print("-" * 50)
    
    # 编译模型
    if config['optimizer'] == 'adam':
        optimizer = Adam(learning_rate=config['learning_rate'])
    elif config['optimizer'] == 'sgd':
        optimizer = SGD(learning_rate=config['learning_rate'], momentum=0.9)
    elif config['optimizer'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=config['learning_rate'])
    else:
        optimizer = Adam(learning_rate=config['learning_rate'])
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # 创建回调
    callbacks_list = create_callbacks(config['patience'], 
                                    f"{model_name}_{config['name']}")
    
    # 训练模型
    start_time = time.time()
    
    history = model.fit(
        x_train, y_train,
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        validation_data=(x_val, y_val),
        callbacks=callbacks_list,
        verbose=1
    )
    
    training_time = time.time() - start_time
    
    # 收集训练结果
    results = {
        'model_name': model_name,
        'config_name': config['name'],
        'history': history.history,
        'training_time': training_time,
        'final_epoch': len(history.history['loss']),
        'best_val_accuracy': max(history.history['val_accuracy']),
        'best_val_loss': min(history.history['val_loss'])
    }
    
    print(f"Training completed in {training_time:.2f} seconds")
    print(f"Best validation accuracy: {results['best_val_accuracy']:.4f}")
    
    return results


5. Training Multiple Models

In [5]:
def train_all_models(models_dict: Dict[str, models.Model],
                    training_configs: List[Dict[str, Any]]) -> Dict[str, Any]:
    """训练所有模型"""
    
    training_results = {}
    
    for model_name, model in models_dict.items():
        model_results = {}
        
        for config in training_configs:
            # 创建模型副本用于训练
            model_copy = models.clone_model(model)
            model_copy.build(model.input_shape)
            
            try:
                results = compile_and_train_model(
                    model_copy, x_train, y_train, x_val, y_val, config, model_name
                )
                model_results[config['name']] = results
                
            except Exception as e:
                print(f"Error training {model_name} with {config['name']}: {e}")
                continue
        
        training_results[model_name] = model_results
    
    return training_results

# 开始训练
print("Starting model training...")
training_results = train_all_models(models_dict, training_configs)

print(f"\nTraining completed for {len(training_results)} models")


Starting model training...

Training simple_cnn with adam_fast configuration...
--------------------------------------------------


2025-09-26 11:08:52.147397: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.
2025-09-26 11:08:52.607476: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.


Epoch 1/50


I0000 00:00:1758856135.351927   13799 service.cc:148] XLA service 0x7f875c010e50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758856135.352051   13799 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-09-26 11:08:55.427215: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1758856135.716319   13799 cuda_dnn.cc:529] Loaded cuDNN version 90101


  3/625 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.1328 - loss: 3.7559   

...

625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.6991 - loss: 0.8648 - val_accuracy: 0.6982 - val_loss: 0.8690 - learning_rate: 0.0010
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7204 - loss: 0.8172
Epoch 6: val_accuracy improved from 0.69820 to 0.73320, saving model to ../models/simple_cnn_adam_fast_best.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7260 - loss: 0.7994 - val_accuracy: 0.7332 - val_loss: 0.7928 - learning_rate: 0.0010
Epoch 7/50
623/625 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7420 - loss: 0.7599
Epoch 7: val_accuracy improved from 0.73320 to 0.74780, saving model to ../models/simple_cnn_adam_fast_best.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.7486 - loss: 0.7426 - val_accuracy: 0.7478 - val_loss: 0.7359 - learning_rate: 0.0010
Epoch 8/50
622/625 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7610 - loss: 0.7127
Epoch 8: val_accuracy did not improve from 0.74780
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7631 - loss: 0.6993 - val_accuracy: 0.7412 - val_loss: 0.7539 - learning_rate: 0.0010
Epoch 9/50
621/625 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7734 - loss: 0.6680
Epoch 9: val_accuracy improved from 0.74780 to 0.78150, saving model to ../models/simple_cnn_adam_fast_best.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.7814 - loss: 0.6518 - val_accuracy: 0.7815 - val_loss: 0.6402 - learning_rate: 0.0010
Epoch 10/50
620/625 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7884 - loss: 0.6254
Epoch 10: val_accuracy did not improve from 0.78150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.7921 - loss: 0.6143 - val_accuracy: 0.7588 - val_loss: 0.7569 - learning_rate: 0.0010
Epoch 11/50
219/625 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8002 - loss: 0.5984

KeyboardInterrupt: 

6. Training Visualization


In [ ]:
def plot_training_history(history: Dict[str, List[float]], 
                         model_name: str, config_name: str) -> None:
    """绘制训练历史"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 损失曲线
    ax1.plot(history['loss'], label='Training Loss')
    ax1.plot(history['val_loss'], label='Validation Loss')
    ax1.set_title(f'{model_name} - {config_name}\nLoss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 准确率曲线
    ax2.plot(history['accuracy'], label='Training Accuracy')
    ax2.plot(history['val_accuracy'], label='Validation Accuracy')
    ax2.set_title(f'{model_name} - {config_name}\nAccuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def compare_training_results(training_results: Dict[str, Any]) -> None:
    """比较训练结果"""
    
    results_data = []
    
    for model_name, model_results in training_results.items():
        for config_name, results in model_results.items():
            results_data.append({
                'Model': model_name,
                'Config': config_name,
                'Best Val Accuracy': results['best_val_accuracy'],
                'Best Val Loss': results['best_val_loss'],
                'Training Time (s)': results['training_time'],
                'Epochs': results['final_epoch']
            })
    
    df = pd.DataFrame(results_data)
    
    # 显示结果表格
    print("Training Results Comparison:")
    print("=" * 80)
    print(df.round(4))
    
    # 绘制比较图
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.barplot(data=df, x='Model', y='Best Val Accuracy', hue='Config')
    plt.title('Validation Accuracy by Model and Config')
    plt.xticks(rotation=45)
    
    plt.subplot(2, 2, 2)
    sns.barplot(data=df, x='Model', y='Training Time (s)', hue='Config')
    plt.title('Training Time by Model and Config')
    plt.xticks(rotation=45)
    
    plt.subplot(2, 2, 3)
    sns.barplot(data=df, x='Model', y='Best Val Loss', hue='Config')
    plt.title('Validation Loss by Model and Config')
    plt.xticks(rotation=45)
    
    plt.subplot(2, 2, 4)
    sns.barplot(data=df, x='Model', y='Epochs', hue='Config')
    plt.title('Training Epochs by Model and Config')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return df

# 可视化训练结果
for model_name, model_results in training_results.items():
    for config_name, results in model_results.items():
        plot_training_history(results['history'], model_name, config_name)

# 比较结果
results_df = compare_training_results(training_results)

7. Hyperparameter Tuning

In [ ]:
def hyperparameter_tuning(model: models.Model,
                         x_train: np.ndarray, y_train: np.ndarray,
                         x_val: np.ndarray, y_val: np.ndarray,
                         param_grid: Dict[str, List]) -> pd.DataFrame:
    """超参数调优"""
    
    results = []
    
    for lr in param_grid['learning_rate']:
        for bs in param_grid['batch_size']:
            for opt in param_grid['optimizer']:
                print(f"\nTuning: lr={lr}, bs={bs}, opt={opt}")
                
                # 创建模型副本
                model_copy = models.clone_model(model)
                model_copy.build(model.input_shape)
                
                # 编译
                if opt == 'adam':
                    optimizer = Adam(learning_rate=lr)
                elif opt == 'sgd':
                    optimizer = SGD(learning_rate=lr, momentum=0.9)
                
                model_copy.compile(
                    optimizer=optimizer,
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                )
                
                # 简短训练进行测试
                history = model_copy.fit(
                    x_train, y_train,
                    batch_size=bs,
                    epochs=10,  # 简短的epochs用于调优
                    validation_data=(x_val, y_val),
                    verbose=0
                )
                
                best_val_acc = max(history.history['val_accuracy'])
                best_val_loss = min(history.history['val_loss'])
                
                results.append({
                    'learning_rate': lr,
                    'batch_size': bs,
                    'optimizer': opt,
                    'val_accuracy': best_val_acc,
                    'val_loss': best_val_loss
                })
    
    return pd.DataFrame(results)

# 超参数网格
param_grid = {
    'learning_rate': [0.001, 0.0005, 0.0001],
    'batch_size': [32, 64, 128],
    'optimizer': ['adam', 'sgd']
}

# 选择最佳模型进行调优
best_model_name = list(models_dict.keys())[0]
best_model = models_dict[best_model_name]

print(f"Performing hyperparameter tuning on {best_model_name}...")
tuning_results = hyperparameter_tuning(best_model, x_train, y_train, x_val, y_val, param_grid)

# 显示最佳参数
best_params = tuning_results.loc[tuning_results['val_accuracy'].idxmax()]
print("\nBest hyperparameters:")
print(best_params)

8. Model Evaluation on Validation Set


In [ ]:
def evaluate_models(models_dict: Dict[str, models.Model],
                   training_results: Dict[str, Any],
                   x_val: np.ndarray, y_val: np.ndarray) -> pd.DataFrame:
    """在验证集上评估模型"""
    
    evaluation_results = []
    
    for model_name, model_results in training_results.items():
        for config_name, results in model_results.items():
            # 加载最佳模型
            model_path = f"../models/{model_name}_{config_name}_best.h5"
            
            try:
                model = models.load_model(model_path)
                
                # 评估模型
                val_loss, val_accuracy = model.evaluate(x_val, y_val, verbose=0)
                
                evaluation_results.append({
                    'Model': model_name,
                    'Config': config_name,
                    'Validation Loss': val_loss,
                    'Validation Accuracy': val_accuracy,
                    'Training Time (s)': results['training_time']
                })
                
            except Exception as e:
                print(f"Error evaluating {model_name}_{config_name}: {e}")
                continue
    
    return pd.DataFrame(evaluation_results)

# 评估模型
print("Evaluating models on validation set...")
eval_df = evaluate_models(models_dict, training_results, x_val, y_val)

# 显示评估结果
print("\nValidation Set Evaluation Results:")
print("=" * 60)
print(eval_df.round(4))

# 找到最佳模型
best_model_row = eval_df.loc[eval_df['Validation Accuracy'].idxmax()]
print(f"\nBest Model: {best_model_row['Model']} with {best_model_row['Config']}")
print(f"Best Validation Accuracy: {best_model_row['Validation Accuracy']:.4f}")

9. Save Training Results

In [ ]:
def save_training_results(training_results: Dict[str, Any],
                         results_df: pd.DataFrame,
                         eval_df: pd.DataFrame,
                         output_dir: str = '../results') -> None:
    """保存训练结果"""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存训练结果到CSV
    results_df.to_csv(os.path.join(output_dir, 'training_results.csv'), index=False)
    eval_df.to_csv(os.path.join(output_dir, 'validation_results.csv'), index=False)
    
    # 保存详细的训练历史
    training_history = {}
    for model_name, model_results in training_results.items():
        training_history[model_name] = {}
        for config_name, results in model_results.items():
            training_history[model_name][config_name] = {
                'history': results['history'],
                'training_time': results['training_time'],
                'best_metrics': {
                    'val_accuracy': results['best_val_accuracy'],
                    'val_loss': results['best_val_loss']
                }
            }
    
    with open(os.path.join(output_dir, 'training_history.json'), 'w') as f:
        json.dump(training_history, f, indent=2)
    
    # 保存最佳模型信息
    best_model_info = eval_df.loc[eval_df['Validation Accuracy'].idxmax()].to_dict()
    with open(os.path.join(output_dir, 'best_model_info.json'), 'w') as f:
        json.dump(best_model_info, f, indent=2)
    
    print(f"Training results saved to {output_dir}")

# 保存结果
save_training_results(training_results, results_df, eval_df)


10. Summary Report


In [ ]:
def generate_training_summary(training_results: Dict[str, Any], 
                            eval_df: pd.DataFrame) -> None:
    """生成训练总结报告"""
    
    print("=" * 60)
    print("MODEL TRAINING SUMMARY REPORT")
    print("=" * 60)
    
    total_models = sum(len(results) for results in training_results.values())
    print(f"Total training runs: {total_models}")
    print(f"Models trained: {len(training_results)}")
    
    # 最佳模型信息
    best_model = eval_df.loc[eval_df['Validation Accuracy'].idxmax()]
    print(f"\nBest Performing Model:")
    print(f"  Model: {best_model['Model']}")
    print(f"  Config: {best_model['Config']}")
    print(f"  Validation Accuracy: {best_model['Validation Accuracy']:.4f}")
    print(f"  Training Time: {best_model['Training Time (s)']:.2f}s")
    
    # 训练统计
    avg_accuracy = eval_df['Validation Accuracy'].mean()
    max_accuracy = eval_df['Validation Accuracy'].max()
    min_accuracy = eval_df['Validation Accuracy'].min()
    
    print(f"\nPerformance Statistics:")
    print(f"  Average Accuracy: {avg_accuracy:.4f}")
    print(f"  Maximum Accuracy: {max_accuracy:.4f}")
    print(f"  Minimum Accuracy: {min_accuracy:.4f}")
    print(f"  Accuracy Range: {max_accuracy - min_accuracy:.4f}")
    
    print(f"\nResults saved to: ../results/")
    print("=" * 60)

# 生成总结
generate_training_summary(training_results, eval_df)

print("Model training completed successfully!")